<a href="https://colab.research.google.com/github/Sopfiee/Game-Recommendation-System/blob/main/Games_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('games_may2024_cleaned.csv', engine='python')

In [13]:
data

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,29018,753,6078,352,1362469,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'...",87,8071426,80,56072
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,590582,"{'Survival': 14611, 'Shooter': 12477, 'Battle ...",58,2365012,68,19517
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,35557,1327,1163,977,668192,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip...",81,2247365,71,23832
3,271590,Grand Theft Auto V,2015-04-13,17,0.00,1,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,12898,590,6910,235,133571,"{'Open World': 31998, 'Action': 23177, 'Multip...",87,1643791,92,16181
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,16213,578,3016,331,68162,"{'FPS': 9689, 'PvP': 9073, 'e-sports': 9026, '...",85,1091695,77,10566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83641,1745960,Circuit Breaker,2021-10-21,0,4.99,1,Circuit Breaker is a turn-based strategy game ...,Circuit Breaker is a turn-based strategy game ...,"In the cyber-future of 2009, the most popular ...",NaN,...,0,0,0,0,0,"{'Strategy': 72, 'Casual': 48, 'PvE': 47, 'Sin...",-1,-1,-1,-1
83642,1888920,Chess vs Chat,2022-08-09,0,3.99,0,Chess Vs Chat allows streamers to host chess m...,Chess Vs Chat allows streamers to host chess m...,Allow your chatters to actively participate an...,NaN,...,0,0,0,0,0,"{'Indie': 31, 'Strategy': 21, 'Puzzle': 11, 'M...",-1,-1,-1,-1
83643,2166420,Orbtangle,2022-10-28,0,1.99,0,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,NaN,...,0,0,0,0,0,"{'Puzzle': 54, 'Colorful': 46, 'Relaxing': 44,...",-1,-1,-1,-1
83644,1456260,Flashlight,2021-10-25,0,2.99,0,Play with Friends Play with or against your fr...,Play with Friends Play with or against your fr...,"Salvage scrap, build defenses, and rank-up to ...",NaN,...,0,0,0,0,0,"{'Top-Down Shooter': 138, 'Rogue-lite': 132, '...",-1,-1,-1,-1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83646 entries, 0 to 83645
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AppID                     83646 non-null  int64  
 1   name                      83643 non-null  object 
 2   release_date              83646 non-null  object 
 3   required_age              83646 non-null  int64  
 4   price                     83646 non-null  float64
 5   dlc_count                 83646 non-null  int64  
 6   detailed_description      83480 non-null  object 
 7   about_the_game            83458 non-null  object 
 8   short_description         83537 non-null  object 
 9   reviews                   10283 non-null  object 
 10  header_image              83646 non-null  object 
 11  website                   39760 non-null  object 
 12  support_url               42081 non-null  object 
 13  support_email             73067 non-null  object 
 14  window

In [20]:
data = data.dropna(subset=['name', 'detailed_description'])

Будем использовать для контентной рекомендательной системы столбцы name, genres, categories, tags, detailed_description

In [16]:
data['tags']

,tags
0,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'..."
1,"{'Survival': 14611, 'Shooter': 12477, 'Battle ..."
2,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip..."
3,"{'Open World': 31998, 'Action': 23177, 'Multip..."
4,"{'FPS': 9689, 'PvP': 9073, 'e-sports': 9026, '..."
...,...
83641,"{'Strategy': 72, 'Casual': 48, 'PvE': 47, 'Sin..."
83642,"{'Indie': 31, 'Strategy': 21, 'Puzzle': 11, 'M..."
83643,"{'Puzzle': 54, 'Colorful': 46, 'Relaxing': 44,..."
83644,"{'Top-Down Shooter': 138, 'Rogue-lite': 132, '..."


Для обработки тегов создадим множество set, чтобы определить уникальные теги

In [21]:
def tags_dict_to_list(x):
  tag_dict=eval(x)
  if type(tag_dict)!=type(list()):
    max_count = sum(tag_dict.values())
  else:
    return ["no_tags"]

  list_tags=[f"{tag}_{count / max_count}"
                      for tag, count in tag_dict.items()]
  return list_tags

In [22]:
import re

In [23]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.split()

In [24]:
data["tags_processed"] = data["tags"].apply(tags_dict_to_list)

In [25]:
def to_low(x):
  res=[i.lower() for i in eval(x)]
  return res

In [26]:
data['genres_processed']= data['genres'].apply(to_low)

In [27]:
data['categories_processed']=data['categories'].apply(to_low)

In [28]:
data['description_processed']=data['detailed_description'].apply(clean_text)

In [29]:
data["data_to_recommend"] = data.apply(lambda row: row["tags_processed"] + row["genres_processed"] +
                           row["categories_processed"] + row["description_processed"] +
                           row["name"].lower().split(), axis=1)

In [63]:
data["data_to_recommend"]

,data_to_recommend
0,"[FPS_0.11572299248692149, Shooter_0.0832322682..."
1,"[Survival_0.12970722440210927, Shooter_0.11076..."
2,"[Free to Play_0.30531960158096283, MOBA_0.1023..."
3,"[Open World_0.10968661945276667, Action_0.0794..."
4,"[FPS_0.07254526123481933, PvP_0.06793303283966..."
...,...
83641,"[Strategy_0.13688212927756654, Casual_0.091254..."
83642,"[Indie_0.2627118644067797, Strategy_0.17796610..."
83643,"[Puzzle_0.1125, Colorful_0.09583333333333334, ..."
83644,"[Top-Down Shooter_0.0659025787965616, Rogue-li..."


In [30]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import ast


In [31]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [32]:
word2vec_model = Word2Vec(sentences=data["data_to_recommend"], vector_size=100, window=5, min_count=2, workers=4)

In [33]:
def get_game_vector(words, model, vector_size=100):
    """Средний вектор по всем словам игры."""
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

In [34]:
data["game_vector"] = data["data_to_recommend"].apply(lambda words: get_game_vector(words, word2vec_model))

In [44]:
data=data.reset_index()

In [41]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')

In [55]:
game_vect=np.vstack(data['game_vector'])
knn.fit(game_vect)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [56]:
def find_similar_games(game_name, df, knn_model, game_vectors, top_k=10):
    # Ищем индекс игры по названию
    game_idx = df[df["name"] == game_name].index

    if len(game_idx) == 0:
        print(f"Игра '{game_name}' не найдена в базе!")
        return []

    game_idx = game_idx[0]  # Берем первый индекс, если найдено несколько

    # Получаем k+1 ближайших соседей (первая — сама игра)

    distances, indices = knn_model.kneighbors([game_vectors[game_idx]], n_neighbors=top_k+1)

    # Пропускаем саму игру (первый элемент)
    similar_games = df.iloc[indices[0][1:]]  # Берем игры с 1 по k

    print(f"Похожие игры на '{game_name}':")
    return similar_games[["name", "genres", "categories"]]  # Можно добавить больше ин

In [61]:
similar_games = find_similar_games("Portal 2", data, knn, game_vect, top_k=10)
similar_games

Похожие игры на 'Portal 2':


,name,genres,categories
7633,Super Motherload,"['Adventure', 'Casual', 'Indie', 'RPG']","['Single-player', 'Multi-player', 'Shared/Spli..."
3689,DG2: Defense Grid 2,"['Indie', 'Strategy']","['Single-player', 'Multi-player', 'Co-op', 'Sh..."
9833,Shadow Complex Remastered,['Action'],"['Single-player', 'Steam Achievements', 'Full ..."
24799,Slime-san: Superslime Edition,"['Action', 'Adventure', 'Indie']","['Single-player', 'Multi-player', 'PvP', 'Shar..."
19665,Q.U.B.E. 10th Anniversary,"['Action', 'Adventure', 'Indie', 'Simulation']","['Single-player', 'Steam Achievements', 'Full ..."
16630,Lode Runner Legacy,"['Action', 'Casual']","['Single-player', 'Multi-player', 'Co-op', 'Sh..."
32771,Sentinels of Earth-Prime,"['Indie', 'Strategy']","['Single-player', 'Multi-player', 'Co-op', 'On..."
619,Bastion,"['Action', 'Indie', 'RPG']","['Single-player', 'Steam Achievements', 'Full ..."
36538,DROD: The Second Sky,"['Adventure', 'Indie', 'Strategy']","['Single-player', 'Steam Achievements', 'Capti..."
4157,Project Arrhythmia,"['Action', 'Adventure', 'Indie', 'Strategy', '...","['Single-player', 'Multi-player', 'Co-op', 'Sh..."


,game_vector
0,"[-0.69119877, -1.0299934, 0.030941715, 1.30078..."
1,"[-0.4382814, -1.0666007, -0.05473601, 1.314971..."
2,"[-0.36954808, -1.0471274, 0.27404898, 1.047231..."
3,"[-0.48437387, -0.9996912, 0.22573283, 1.219761..."
4,"[-0.48074594, -1.2134991, 0.4518339, 1.2111967..."
...,...
83474,"[-0.36860672, -1.2845993, 0.7187198, 1.0861993..."
83475,"[-0.102190115, -1.031841, 0.46923202, 0.945740..."
83476,"[-0.41808993, -0.9788508, 0.21899223, 1.270238..."
83477,"[-0.7063461, -1.2425845, 0.67662114, 1.0289286..."


32

,recommendations
0,4111974.0
1,1701431.0
2,14329.0
3,1641404.0
4,1088708.0
...,...
17926,121.0
17927,122.0
17928,122.0
17929,122.0
